In [1]:
import mtt
import numpy as np
from copy import copy, deepcopy

In [2]:
# Default simulation values
obj1 = np.array([[0], [0], [1], [1]])
obj2 = np.array([[1], [1], [1], [1]])
initial = {0: obj1}
dt = 1
ep_normal = 1
ep_tangent = 1
nu = 1
ts = 6
miss_p = 0.01
lam = 0.01
fa_scale = 10
gen = mtt.MultiObjSimple(initial, dt, ep_tangent, ep_normal, nu)
params = gen.get_params()
P = params.pop("P")

In [3]:
params

{'f': <bound method MultiObjSimple.process_function of <mtt.multiobject_simple.MultiObjSimple object at 0x16a64bdf0>>,
 'A': <bound method MultiObjSimple.process_jacobian of <mtt.multiobject_simple.MultiObjSimple object at 0x16a64bdf0>>,
 'h': <bound method MultiObjSimple.measurement_function of <mtt.multiobject_simple.MultiObjSimple object at 0x16a64bdf0>>,
 'Q': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'W': <bound method MultiObjSimple.W of <mtt.multiobject_simple.MultiObjSimple object at 0x16a64bdf0>>,
 'R': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'H': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])}

In [4]:
k = mtt.KalmanFilter(**params)

In [5]:
xt, P = k.time_update(obj1, P)
k.measurement_update(xt, P, obj2)

(array([[1.],
        [1.],
        [1.],
        [1.]]),
 array([[ 6.25000000e-01,  1.58539631e-19,  1.25000000e-01,
         -4.75618892e-19],
        [ 1.58539631e-19,  6.25000000e-01, -4.75618892e-19,
          1.25000000e-01],
        [ 1.25000000e-01, -4.75618892e-19,  6.25000000e-01,
          1.42685668e-18],
        [-4.75618892e-19,  1.25000000e-01,  1.42685668e-18,
          6.25000000e-01]]))

In [6]:
gate = mtt.DistanceGatingMHT(0.9)

tot = -1000
tmm = -2000
tnt = -3000
main = mtt.TrackMaintenanceMHT(tot, tmm, tnt, 1-miss_p, 4, lam, params['R'], k)
hypo = mtt.HypothesisComp()
prune = mtt.Pruning(4)

tracker = mtt.MHTTracker(k, gate, main, hypo, prune)

In [7]:
tracker.measurements

[]

In [8]:
# test = np.zeros(4)
# test.shape = (4,1)
# tracker.predict([test])

In [9]:
tracker.tracks

[]

In [10]:
tracker.get_trajectories()

[]

In [11]:
tracker.tracks

[]

In [12]:
processes = dict()
measures = dict()
measure_colors = dict()
cur_seed = np.random.randint(10**7)
rng = np.random.default_rng(cur_seed)
process = gen.process(5, rng)
processes[len(processes.keys())] = process
measures[len(measures.keys())], measure_colors[len(measure_colors.keys())] = gen.measure(process, rng)
print(processes[0])

[{0: array([[0],
       [0],
       [1],
       [1]])}, {0: array([[ 1.        ],
       [ 1.        ],
       [ 1.02428975],
       [-0.6598427 ]])}, {0: array([[2.02428975],
       [0.3401573 ],
       [0.88472568],
       [0.87557323]])}, {0: array([[2.90901544],
       [1.21573053],
       [1.33344207],
       [0.41268345]])}, {0: array([[4.2424575 ],
       [1.62841398],
       [0.18019455],
       [0.38558545]])}, {0: array([[ 4.42265205],
       [ 2.01399942],
       [-0.2467969 ],
       [ 0.23506988]])}]


In [13]:
for i in range(len(processes[0])):
    tracker.predict(deepcopy(measures[0][i]))

([0], 1000)
NODES:  [0]
[0]
--------
([1, 2], 1032)
NODES:  [0, 1, 2]
[1, 2]
--------
([6], 1000)
NODES:  [0, 1, 2, 3, 4, 5, 6]
[6]
--------
([14], 1000)
NODES:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[14]
--------
([30], 1000)
NODES:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[30]
--------
([62], 1000)
NODES:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[62]
--------


In [14]:
import networkx as nx
import networkx.algorithms.clique as nxac
g = nx.Graph()
g.add_node(0, weight = 1)
print(nxac.max_weight_clique(g))

([0], 1)
